In [1]:
using Agents
using Random
using InteractiveDynamics
using CairoMakie

@agent tcell ContinuousAgent{2} begin
    activation_status::Float64
    has_been_activated::Bool
    time_after_activation::Int
end

@agent dcell ContinuousAgent{2} begin
        activation_status::Float64
        time_after_activation::Int
end

In [24]:
cells = Union{tcell, dcell}

function initialize_model(; 
    n_dcell = 30, 
    n_tcell = 30, 
    speed = 2,
    extent = (300,300),    
    visual_distance = 5.0,
    activation_status = 0.0,
     has_been_activated = false,
    time_after_activation = 0
    )

    space2d = ContinuousSpace(extent; spacing = 10,)
    
    model = ABM(cells, space2d ,rng = MersenneTwister(30),properties = Dict(:dt => 1.0); 
        scheduler = Schedulers.randomly)

    #scale the random number
    
    for _ in 1:n_tcell
        pos = Tuple(rand(model.rng, 2)).*300
        vel = sincos(2π * rand(model.rng)) .* speed 
        add_agent!(
            pos, tcell, model , vel, activation_status, has_been_activated, time_after_activation
        )
    end
    
    for _ in 1:n_dcell
        pos = Tuple(rand(model.rng, 2)).*300
        vel = sincos(2π * rand(model.rng)) .* speed .* 0.6
        add_agent!(
            pos, dcell, model, vel, activation_status, time_after_activation
        )
    end

    return model
end

initialize_model (generic function with 1 method)

In [25]:
model = initialize_model()

function agent_step!(cells,model;
        speed = 5,
       )
     
#     tcell.vel = sincos(2π * rand(model.rng)) .* speed 
#     dcell.vel = sincos(2π * rand(model.rng)) .* speed  .* 0.6
    
    if cells isa dcell
        cells.vel = sincos(2π * rand(model.rng)) .* speed .* 0.6
        neighbor_ids = nearby_ids(cells,model,5)
        
        for id in neighbor_ids
            neighbor = model[id]
            if neighbor isa tcell && neighbor.has_been_activated == false
#                 print(typeof(model[id])) 
                cells.vel = (0,0)
            end
        end
        
        
    else 
        cells.vel = sincos(2π * rand(model.rng)) .* speed 
        neighbor_ids = nearby_ids(cells,model,5)
    
      
        for id in neighbor_ids
            neighbor = model[id]
            if neighbor isa dcell && cells.has_been_activated == false
                   cells.has_been_activated = true
                   cells.vel = (0,0)
                   cells.activation_status += 1/240
                   cells.time_after_activation += 1
                    break
                
            elseif neighbor isa dcell && cells.has_been_activated == true &&cells.time_after_activation< 1440
                    cells.time_after_activation += 1
                    cells.vel = (0,0)
                    cells.activation_status += 1/240
                    break
                
            elseif cells.time_after_activation == 1440
                     cells.vel = sincos(2π * rand(model.rng)) .* speed 
                    cells.activation_status -= 1/60

                    break


                   
                                               
  end
    
                
        end
    end
    


    
    move_agent!(cells, model, model.dt)
end

CairoMakie.activate!()



┌ Warning: AgentType is not concrete. If your agent is parametrically typed, you're probably
│ seeing this warning because you gave `Agent` instead of `Agent{Float64}`
│ (for example) to this function. You can also create an instance of your agent
│ and pass it to this function. If you want to use `Union` types for mixed agent
│ models, you can silence this warning.
└ @ Agents /Users/kimchanghyeon/.julia/packages/Agents/QtIUJ/src/core/model.jl:299


In [26]:
# offset(a) = a isa tcell ? (-0.1, -0.1*rand()) : (+0.1, +0.1*rand())
ashape(a) = :circle
asize(a) = a isa dcell ? 20 : 10
acolor(a) = a.activation_status > 0 ? :blue : :black
# ac(dcell) = :blue
# ac(tcell) = :black

acolor (generic function with 1 method)

In [27]:
abmvideo("tcell.mp4", model, agent_step!;
title = "immune cell", framerate  = 100, 
    frames = 2880, as=asize, am = ashape, ac = acolor)

In [ ]:
tcell dcell detach 될때 다시 값 감소 해서 색깔 바뀌는 것 구현

   